In [465]:
import numpy as np
from scipy.stats import beta

# parameters
sigma = 5               # elasticity of substitution between critical goods
theta = 0.1             # between outside and critical good
kappa = 1               # fixed entry cost
z_h = {"A": 0, "B": 0}  # productivity thresholds for countries A and B
e_h = {"A": 0, "B": 0}  # entry subsidy for countries A and B
t_Fij = {"A": {"A": 1.0, "B": 1.1},  # trade costs for final goods from A to (A, B)
         "B": {"A": 1.1, "B": 1.0}}  # trade costs for final goods from B to (A, B)

alpha_c = 4  # alpha parameter for country risk
beta_c = 1   # beta parameter (i.e. 1)
alpha_i = 4  # alpha parameter for idiosyncratic risk
beta_i = 1   # beta parameter

cvar_alpha = 0.95  # confidence level for CVaR

# outside good consumption; fix this later
x_0j = 1

# expected survival probability
expected_phi = (alpha_c/(alpha_c + beta_c))*(alpha_i/(alpha_i + beta_i))

# trade costs for inputs
t_Ihi = {"A": {"A": 1.0, "B": 1.9},  # trade costs for inputs from A to (A, B)
         "B": {"A": 1.9, "B": 1.0}}  # trade costs for inputs from B to (A, B)

# calculate input prices
p_Ih = {
    "A": (sigma / (sigma - 1)) * (1 / expected_phi - z_h["A"]),
    "B": (sigma / (sigma - 1)) * (1 / expected_phi - z_h["B"]),
}

# calculate price of final goods (p_Fi) = input price index (bar_P_Ii)
p_Fi = {}
for i in ("A", "B"):
    sum_term = 0
    for h in ("A", "B"):
        term = (p_Ih[h] * t_Ihi[h][i]) ** (1 - sigma)
        sum_term += term
    p_Fi[i] = sum_term ** (1 / (1 - sigma))

# calculate price index for final goods consumed in country j
bar_P_Fj = {}
for j in ("A", "B"):
    sum_term = 0
    for i in ("A", "B"):
        term = (p_Fi[i] * t_Fij[i][j]) ** (1 - sigma)
        sum_term += term
    bar_P_Fj[j] = sum_term ** (1 / (1 - sigma))

# expected demand for final goods
expected_x_Fij = {}
for i in ("A", "B"):
    expected_x_Fij[i] = {}
    for j in ("A", "B"):
        expected_x_Fij[i][j] = (
            (p_Fi[i] * t_Fij[i][j]) ** (-sigma)
            * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta))
        )

# output results
print(f"Price of final goods (made by A): {p_Fi['A']:.3f}")
print(f"Price of final goods (made by B): {p_Fi['B']:.3f}")
print(f"Price index for final goods (sold to A): {bar_P_Fj['A']:.3f}")
print(f"Price index for final goods (sold to B): {bar_P_Fj['B']:.3f}")
print("")

# monte carlo setup
num_simulations = 45000
max_iterations = 1000
iteration = 0
results = []
sample_variances = []  # store variance of U_j for each iteration
cvar_results = []  # store CVaR for each iteration
tolerance = 0.01  # relative error tolerance (e.g., 1%)
z_alpha = 1.96  # critical z-value for 95% confidence

# run the loop until convergence or max iterations
while iteration < max_iterations:
    iteration += 1   
    
    D_h_A = 1 - beta.rvs(alpha_c, beta_c, size=num_simulations)
    D_h_B = 1 - beta.rvs(alpha_c, beta_c, size=num_simulations)
    
    fail_all_A = np.random.rand(num_simulations) < D_h_A
    fail_all_B = np.random.rand(num_simulations) < D_h_B
    
    phi_A_individual = beta.rvs(alpha_i, beta_i, size=num_simulations)
    phi_B_individual = beta.rvs(alpha_i, beta_i, size=num_simulations)
    
    phi_A = np.where(fail_all_A, 0, phi_A_individual)
    phi_B = np.where(fail_all_B, 0, phi_B_individual)
    
    U_j = np.zeros(num_simulations)  # Placeholder for new utility values

    # compute rho_i(omega) for A and B
    rho_i = {}
    for i in ("A", "B"):
        rho_i[i] = np.zeros(num_simulations)
        for omega in range(num_simulations):
            sum_term = 0
            for h in ("A", "B"):
                term = (
                    (phi_A[omega] if h == "A" else phi_B[omega]) / expected_phi
                ) ** ((sigma - 1) / sigma) * p_Ih[h] ** (1 - sigma) * t_Ihi[h][i] ** (1 - sigma)
                sum_term += term
            rho_i[i][omega] = p_Fi[i] ** sigma * sum_term ** (sigma / (sigma - 1))
    
    for omega in range(num_simulations):
        sum_term = sum(
            [
                (rho_i["A"][omega] * sum(expected_x_Fij["A"].values())) ** ((sigma - 1) / sigma),
                (rho_i["B"][omega] * sum(expected_x_Fij["B"].values())) ** ((sigma - 1) / sigma),
            ]
        )
        U_j[omega] = x_0j + (1 / theta) * (sum_term ** (theta * sigma / (sigma - 1)))
    
    # compute and store the sample mean and variance
    sample_mean = np.mean(U_j)
    sample_variance = np.var(U_j, ddof=1)
    results.append(sample_mean)
    sample_variances.append(sample_variance)  # Store variance of this iteration

    # calculate VaR and CVaR
    sorted_U_j = np.sort(U_j)
    var_threshold = sorted_U_j[int(np.ceil((1 - cvar_alpha) * num_simulations)) - 1]  # VaR at 1-alpha
    cvar = np.mean(sorted_U_j[sorted_U_j <= var_threshold])  # Mean of values below VaR
    cvar_results.append(cvar)
    
    # calculate mean and CVaR statistics
    mean_U_j = np.mean(results)
    overall_variance = np.var(results, ddof=1)
    average_cvar = np.mean(cvar_results)
    overall_cvar_variance = np.var(cvar_results, ddof=1)
    cvar_sem = np.std(cvar_results, ddof=1) / np.sqrt(len(cvar_results))  # SEM of CVaR
    mean_sem = np.std(results, ddof=1) / np.sqrt(len(results))  # SEM of mean utility

    # calculate CI widths
    mean_ci_width = 2 * z_alpha * mean_sem
    cvar_ci_width = 2 * z_alpha * cvar_sem
    
    # check convergence criteria
    mean_converged = (mean_ci_width / mean_U_j) < tolerance
    cvar_converged = (cvar_ci_width / average_cvar) < tolerance
    
    if mean_converged and cvar_converged:
        print(f"Converged after {iteration} iterations.")
        print("")
        break
else:
    print("Reached maximum iterations without convergence.")
    print("")

# output final results
average_sample_variance = np.mean(sample_variances) # average variance per sample
print(f"Overall mean utility: {mean_U_j:.3f}")
print(f"Mean CVaR (at α = {cvar_alpha}): {average_cvar:.3f}")
print("")
print(f"Mean sample variance of utility: {average_sample_variance:.3f}")
print(f"Variance of mean utility: {overall_variance:.3f}")
print(f"Variance of mean CVaR: {overall_cvar_variance:.3f}")

Price of final goods (made by A): 1.917
Price of final goods (made by B): 1.917
Price index for final goods (sold to A): 1.683
Price index for final goods (sold to B): 1.683

Converged after 513 iterations.

Overall mean utility: 10.644
Mean CVaR (at α = 0.95): 2.740

Mean sample variance of utility: 4.070
Variance of mean utility: 0.000
Variance of mean CVaR: 0.025


# Notes

## high tariff case (on final goods)

Price of final goods (made by A): 1.715

Price of final goods (made by B): 1.715

Price index for final goods (sold to A): 1.683

Price index for final goods (sold to B): 1.683

Converged after 608 iterations.

Overall mean utility: 10.755

Mean CVaR (at α = 0.95): 2.753

Mean sample variance of utility: 4.169

Variance of mean utility: 0.000

Variance of mean CVaR: 0.030

## high tariff case (on inputs)

Price of final goods (made by A): 1.917

Price of final goods (made by B): 1.917

Price index for final goods (sold to A): 1.683

Price index for final goods (sold to B): 1.683

Converged after 513 iterations.

Overall mean utility: 10.644

Mean CVaR (at α = 0.95): 2.740

Mean sample variance of utility: 4.070

Variance of mean utility: 0.000

Variance of mean CVaR: 0.025

## no tariff case

Price of final goods (made by A): 1.715

Price of final goods (made by B): 1.715

Price index for final goods (sold to A): 1.506

Price index for final goods (sold to B): 1.506

Converged after 576 iterations.

Overall mean utility: 10.764

Mean CVaR (at α = 0.95): 2.749

Mean sample variance of utility: 4.179

Variance of mean utility: 0.000

Variance of mean CVaR: 0.028